<a href="https://colab.research.google.com/github/kimtaewan22/colabtest/blob/main/%EC%9D%91%EC%9A%A9%EC%88%98%ED%95%99_2023%EB%85%84_1%ED%95%99%EA%B8%B0_2021040031_%EA%B9%80%ED%83%9C%EC%99%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# train파일 로드 및 수정

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# 데이터 준비
wine_data = pd.read_csv('train.csv')
# 사용하지 않는 feature는 드랍
wine_data.drop(['index','quality'],axis=1, inplace=True)

print(wine_data.describe())

       fixed acidity  volatile acidity  citric acid  residual sugar  \
count    5497.000000       5497.000000  5497.000000     5497.000000   
mean        7.210115          0.338163     0.318543        5.438075   
std         1.287579          0.163224     0.145104        4.756676   
min         3.800000          0.080000     0.000000        0.600000   
25%         6.400000          0.230000     0.250000        1.800000   
50%         7.000000          0.290000     0.310000        3.000000   
75%         7.700000          0.400000     0.390000        8.100000   
max        15.900000          1.580000     1.660000       65.800000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  5497.000000          5497.000000           5497.000000  5497.000000   
mean      0.055808            30.417682            115.566491     0.994673   
std       0.034653            17.673881             56.288223     0.003014   
min       0.009000             1.000000         

#라벨 인코딩 및 이상치 개수 확인


In [ ]:
from sklearn.preprocessing import LabelEncoder

# type을 수치형 데이터로 변환하기 위한 라벨 인코딩 진행
label_encoder = LabelEncoder()
wine_data['type_encoded'] = label_encoder.fit_transform(wine_data['type'])

# 이상치 개수 확인 함수 정의
def count_outliers(feature, subset):
    Q1 = subset[feature].quantile(0.25)
    Q3 = subset[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - (1.5 * IQR)
    upper_bound = Q3 + (1.5 * IQR)
    outliers = subset[(subset[feature] < lower_bound) | (subset[feature] > upper_bound)]
    return outliers.shape[0]

# 각 특성별로 이상치 개수 출력
features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
            'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
            'pH', 'sulphates', 'alcohol']

# 화이트 와인 데이터의 이상치 개수 확인
white_wine_outliers = wine_data[wine_data['type_encoded'] == 1]
for feature in features:
    outlier_count = count_outliers(feature, white_wine_outliers)
    print(f"화이트 와인 이상치 개수 ({feature}): {outlier_count}")

# 레드 와인 데이터의 이상치 개수 확인
red_wine_outliers = wine_data[wine_data['type_encoded'] == 0]
for feature in features:
    outlier_count = count_outliers(feature, red_wine_outliers)
    print(f"레드 와인 이상치 개수 ({feature}): {outlier_count}")


화이트 와인 이상치 개수 (fixed acidity): 99
화이트 와인 이상치 개수 (volatile acidity): 154
화이트 와인 이상치 개수 (citric acid): 231
화이트 와인 이상치 개수 (residual sugar): 8
화이트 와인 이상치 개수 (chlorides): 172
화이트 와인 이상치 개수 (free sulfur dioxide): 38
화이트 와인 이상치 개수 (total sulfur dioxide): 16
화이트 와인 이상치 개수 (density): 5
화이트 와인 이상치 개수 (pH): 64
화이트 와인 이상치 개수 (sulphates): 99
화이트 와인 이상치 개수 (alcohol): 0
레드 와인 이상치 개수 (fixed acidity): 39
레드 와인 이상치 개수 (volatile acidity): 17
레드 와인 이상치 개수 (citric acid): 1
레드 와인 이상치 개수 (residual sugar): 125
레드 와인 이상치 개수 (chlorides): 96
레드 와인 이상치 개수 (free sulfur dioxide): 25
레드 와인 이상치 개수 (total sulfur dioxide): 43
레드 와인 이상치 개수 (density): 37
레드 와인 이상치 개수 (pH): 26
레드 와인 이상치 개수 (sulphates): 58
레드 와인 이상치 개수 (alcohol): 9


# MLP-pytorch





#데이터 로드 및 분할

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# 시드 값을 주어 모델의 초기화나 데이터 배치의 순서 등이 동일하게 유지되어 실험 결과의 재현성을 확보.
torch.manual_seed(42)

# 데이터 로드
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# 학습 데이터에서 선택한 특성과 분류 대상을 이용 데이터분할
X = train_data[['fixed acidity', 'volatile acidity',
                    'chlorides', 'free sulfur dioxide', 'total sulfur dioxide',
                    'sulphates']]
X.loc[:, 'new_feature'] = X['free sulfur dioxide'] / X['total sulfur dioxide']
y = train_data['type']

# 데이터 전처리
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# 데이터를 학습 데이터와 검증 데이터로 8:2 비율로 분할. stratify 옵션을 주어 클래스 비율을 유지하면서 데이터를 분할하도록 함
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# 테스트 데이터에서 선택한 특성과 분류 대상을 이용 데이터분할
X_test = test_data[['fixed acidity', 'volatile acidity',
                    'chlorides', 'free sulfur dioxide', 'total sulfur dioxide',
                    'sulphates']]
X_test.loc[:, 'new_feature'] = X_test['free sulfur dioxide'] / X_test['total sulfur dioxide']
y_test = test_data['type']

<ipython-input-58-ec79d3aa77a2>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:, 'new_feature'] = X['free sulfur dioxide'] / X['total sulfur dioxide']
<ipython-input-58-ec79d3aa77a2>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.loc[:, 'new_feature'] = X_test['free sulfur dioxide'] / X_test['total sulfur dioxide']


#데이터 전처리

In [ ]:
# 데이터 전처리
X_test_scaled = scaler.transform(X_test)

# 라벨 매핑 white와 red를 각각 0 과 1의 수치화 데이터로 매핑
label_mapping = {'white': 0, 'red': 1}
y_train_mapped = y_train.map(label_mapping)
y_val_mapped = y_val.map(label_mapping)
y_test_mapped = y_test.map(label_mapping)

# 입력 데이터를 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

# 타겟 데이터를 텐서로 변환
y_train_tensor = torch.tensor(y_train_mapped.values, dtype=torch.long)
y_val_tensor = torch.tensor(y_val_mapped.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_mapped.values, dtype=torch.long)


#WeightedRandomSampler

In [ ]:
# 클래스별 가중치를 적용하기 위한 WeightedRandomSampler 생성
class_counts = torch.bincount(y_train_tensor) # 각 클래의 개수를 셈
class_weights = 1.0 / class_counts

weights = class_weights[y_train_tensor] # 각 샘플에 대한 가중치를 가져옴
sampler = torch.utils.data.WeightedRandomSampler(weights, len(weights))

# 데이터 로더 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor) #  입력 데이터(X_train_tensor)와 타겟 데이터(y_train_tensor)를 묶어서 TensorDataset으로 생성
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)

# Model

In [ ]:
# 신경망 모델 정의
class WineClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(WineClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
# forward 함수
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
# 모델 초기화
input_size = X_train.shape[1]
hidden_size = 64
num_classes = 2
model = WineClassifier(input_size, hidden_size, num_classes)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train function

In [ ]:
# 학습 함수 정의
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad() # 기울기 값 누적 방지를 위해 매 반복마다 이전 기울기 0으로 초기화
            outputs = model(inputs) # 입력데이터를 모델에 전달하여 예측값을 얻음.
            loss = criterion(outputs, labels) # 손실 함수를 통해 loss 계산
            loss.backward() # 역전파 수행
            optimizer.step() # 최적화 수행 후 파라미터 업데이트
            running_loss += loss.item()
        if (epoch+1) % 100 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}")

# 모델 학습 부분

In [ ]:
# 모델 학습
# num_epochs = 1000
# train(model, train_loader, criterion, optimizer, num_epochs)


Epoch 100/1000, Loss: 1.5380
Epoch 200/1000, Loss: 0.7957
Epoch 300/1000, Loss: 0.3601
Epoch 400/1000, Loss: 0.1227
Epoch 500/1000, Loss: 0.2423
Epoch 600/1000, Loss: 0.1294
Epoch 700/1000, Loss: 0.1541
Epoch 800/1000, Loss: 0.1544
Epoch 900/1000, Loss: 0.0092
Epoch 1000/1000, Loss: 0.0256


# 모델 저장 부분

In [ ]:
# 모델 저장
# torch.save(model.state_dict(), '2021040031_김태완.model')

# 저장된 모델을 로드하는 부분

In [ ]:
# 모델 객체 생성 후 저장된 모델의 상태를 불러와 초기화
model_d = WineClassifier(input_size, hidden_size, num_classes)
model_d.load_state_dict(torch.load('2021040031_김태완.model'))

<All keys matched successfully>

# 검증 데이터 평가

In [ ]:
# 검증 데이터로 모델 평가
model_d.eval()
with torch.no_grad():
    outputs = model_d(X_val_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = torch.sum(predicted == y_val_tensor).item() / len(y_val_tensor)
    print(f"Validation Accuracy: {accuracy:.4f}")

    # 분류 보고서 출력
    target_names = ['white', 'red']
    report = classification_report(y_val_tensor, predicted, digits=3, target_names=target_names)
    print(report)

Validation Accuracy: 0.9909
              precision    recall  f1-score   support

       white      0.995     0.993     0.994       832
         red      0.978     0.985     0.981       268

    accuracy                          0.991      1100
   macro avg      0.986     0.989     0.988      1100
weighted avg      0.991     0.991     0.991      1100



# 테스트 데이터 평가

In [ ]:
# 테스트 데이터로 모델 평가
model_d.eval()
with torch.no_grad():
    outputs = model_d(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = torch.sum(predicted == y_test_tensor).item() / len(y_test_tensor)
    print(f"Test Accuracy: {accuracy:.4f}")

    # 분류 보고서 출력
    target_names = ['white', 'red']
    report = classification_report(y_test_tensor, predicted, digits=3, target_names=target_names)
    print(report)


Test Accuracy: 0.9950
              precision    recall  f1-score   support

       white      0.993     1.000     0.997       739
         red      1.000     0.981     0.990       261

    accuracy                          0.995      1000
   macro avg      0.997     0.990     0.993      1000
weighted avg      0.995     0.995     0.995      1000



# 새로운 Feature를 추가하기 전 모델

*   위의 코드에서 추가 feature만 제외하고 다 동일한 코드





1.   데이터 로딩 및 분할




In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

torch.manual_seed(2022)

# 데이터 로드
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# 학습 데이터에서 선택한 특성과 분류 대상을 이용 데이터분할
X = train_data[['fixed acidity', 'volatile acidity',
                    'chlorides', 'free sulfur dioxide', 'total sulfur dioxide',
                    'sulphates']]
y = train_data['type']

# 데이터 전처리
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

# 데이터를 학습 데이터와 검증 데이터로 8:2 비율로 분할.
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# 테스트 데이터에서 선택한 특성과 분류 대상을 이용 데이터분할
X_test = test_data[['fixed acidity', 'volatile acidity',
                    'chlorides', 'free sulfur dioxide', 'total sulfur dioxide',
                    'sulphates']]
y_test = test_data['type']





2.  데이터 전처리



In [ ]:

# 데이터 전처리
X_test_scaled = scaler.transform(X_test)

# 라벨 매핑
label_mapping = {'white': 0, 'red': 1}
y_train_mapped = y_train.map(label_mapping)
y_val_mapped = y_val.map(label_mapping)
y_test_mapped = y_test.map(label_mapping)

# 입력 데이터를 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

# 타겟 데이터를 텐서로 변환
y_train_tensor = torch.tensor(y_train_mapped.values, dtype=torch.long)
y_val_tensor = torch.tensor(y_val_mapped.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_mapped.values, dtype=torch.long)

3. 신경망 모델 및 학습 함수

In [ ]:
# 클래스별 가중치를 적용하기 위한 WeightedRandomSampler 생성
class_counts = torch.bincount(y_train_tensor)
class_weights = 1.0 / class_counts

weights = class_weights[y_train_tensor]
sampler = torch.utils.data.WeightedRandomSampler(weights, len(weights))

# 데이터 로더 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)

# 신경망 모델 정의
class WineClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(WineClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# 모델 초기화
input_size = X_train.shape[1]
hidden_size = 64
num_classes = 2
model = WineClassifier(input_size, hidden_size, num_classes)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 학습 함수 정의
def train(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad() # 기울기 값 누적 방지를 위해 매 반복마다 이전 기울기 0으로 초기화
            outputs = model(inputs) # 입력데이터를 모델에 전달하여 예측값을 얻음.
            loss = criterion(outputs, labels) # 손실 함수를 통해 loss 계산
            loss.backward() # 역전파 수행
            optimizer.step() # 최적화 수행 후 파라미터 업데이트
            running_loss += loss.item()
        if (epoch+1) % 100 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}")


4. 모델 학습

In [ ]:
# 모델 학습
# num_epochs = 1000
# train(model, train_loader, criterion, optimizer, num_epochs)

5. 모델 저장

In [ ]:
# 모델 저장
# torch.save(model.state_dict(), '2021040031_김태완1.model')

6. 모델 객체 생성

In [ ]:
# 모델 객체 생성 후 저장된 모델의 상태를 불러와 초기화
model_w = WineClassifier(input_size, hidden_size, num_classes)
model_w.load_state_dict(torch.load('2021040031_김태완1.model'))

<All keys matched successfully>

7. 모델평가

In [ ]:
model_w.eval()
with torch.no_grad():
    outputs = model_w(X_val_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = torch.sum(predicted == y_val_tensor).item() / len(y_val_tensor)
    print(f"Validation Accuracy: {accuracy:.4f}")

# 테스트 데이터로 모델 평가
model_w.eval()
with torch.no_grad():
    outputs = model_w(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = torch.sum(predicted == y_test_tensor).item() / len(y_test_tensor)
    print(f"Test Accuracy: {accuracy:.4f}")

    # 분류 보고서 출력
    target_names = ['white', 'red']
    report = classification_report(y_test_tensor, predicted, digits=3, target_names=target_names)
    print(report)

Validation Accuracy: 0.9882
Test Accuracy: 0.9900
              precision    recall  f1-score   support

       white      0.992     0.995     0.993       739
         red      0.985     0.977     0.981       261

    accuracy                          0.990      1000
   macro avg      0.988     0.986     0.987      1000
weighted avg      0.990     0.990     0.990      1000

